# Installing  and importing packages

In [1]:
!pip install tqdm
!pip install mpi4py
!pip install petsc4py
!pip install vtk
!pip install pyvista
!pip install pyvirtualdisplay

In [2]:
import gmsh
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook

import numpy.typing
import typing

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx.cpp.mesh import to_type, cell_entity_type
from dolfinx.fem import Constant, Function, FunctionSpace, assemble_scalar, dirichletbc, form, locate_dofs_topological, set_bc
from dolfinx.fem.petsc import apply_lifting, assemble_matrix, assemble_vector, create_vector, set_bc
from dolfinx.graph import create_adjacencylist
from dolfinx.geometry import BoundingBoxTree, compute_collisions, compute_colliding_cells
from dolfinx.io import (XDMFFile, cell_perm_gmsh, distribute_entity_data, extract_gmsh_geometry,
                        extract_gmsh_topology_and_markers, ufl_mesh_from_gmsh)
from dolfinx.mesh import create_mesh, meshtags_from_entities

import ufl

from ufl import (FacetNormal, FiniteElement, Identity, Measure, TestFunction, TrialFunction, VectorElement,
                 as_vector, div, dot, ds, dx, inner, lhs, grad, nabla_grad, rhs, sym)

# Creating geometry / mesh

In [3]:
gmsh.initialize()

L = 2.2
H = 1
c_x =0.2
c_y = 0.5
r = 0.05
gdim = 2
rank = MPI.COMM_WORLD.rank
if rank == 0:
    rectangle = gmsh.model.occ.addRectangle(0,0,0, L, H, tag=1)
    obstacle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)

if rank == 0:
    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, obstacle)])
    gmsh.model.occ.synchronize()
    
fluid_marker = 1
if rank == 0:
    volumes = gmsh.model.getEntities(dim=gdim)
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

inlet_marker, outlet_marker, wall_marker, obstacle_marker = 2, 3, 4, 5
inflow, outflow, walls, obstacle = [], [], [], []
if rank == 0:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [0, H/2, 0]):
            inflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L, H/2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L/2, H, 0]) or np.allclose(center_of_mass, [L/2, 0, 0]):
            walls.append(boundary[1])
        else:
            obstacle.append(boundary[1])
    gmsh.model.addPhysicalGroup(1, walls, wall_marker)
    gmsh.model.setPhysicalName(1, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

res_min = r / 1 #3.7
res_max = 10 * r # 1.5
if rank == 0:
    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "EdgesList", obstacle)
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "IField", 1)
    gmsh.model.mesh.field.setNumber(2, "LcMin", res_min)
    gmsh.model.mesh.field.setNumber(2, "LcMax", res_max)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 4*r)
    gmsh.model.mesh.field.setNumber(2, "DistMax", 8*r)

    # We take the minimum of the two fields as the mesh size
    gmsh.model.mesh.field.add("Min", 5)
    gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2])
    gmsh.model.mesh.field.setAsBackgroundMesh(5)

if rank == 0:
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 8)
    gmsh.option.setNumber("Mesh.RecombineAll", 2)
    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
    gmsh.model.mesh.generate(gdim)
    gmsh.model.mesh.optimize("Netgen")

if MPI.COMM_WORLD.rank == 0:
    # Get mesh geometry
    
    x = extract_gmsh_geometry(gmsh.model)

    # Get mesh topology for each element
    topologies = extract_gmsh_topology_and_markers(gmsh.model)
    # Get information about each cell type from the msh files
    num_cell_types = len(topologies.keys())
    cell_information = {}
    cell_dimensions = np.zeros(num_cell_types, dtype=np.int32)
    for i, element in enumerate(topologies.keys()):
        properties = gmsh.model.mesh.getElementProperties(element)
        name, dim, order, num_nodes, local_coords, _ = properties
        cell_information[i] = {"id": element, "dim": dim, "num_nodes": num_nodes}
        cell_dimensions[i] = dim

    # Sort elements by ascending dimension
    perm_sort = np.argsort(cell_dimensions)

    # Broadcast cell type data and geometric dimension
    cell_id = cell_information[perm_sort[-1]]["id"]
    tdim = cell_information[perm_sort[-1]]["dim"]
    num_nodes = cell_information[perm_sort[-1]]["num_nodes"]
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([cell_id, num_nodes], root=0)
    if tdim - 1 in cell_dimensions:
        num_facet_nodes = MPI.COMM_WORLD.bcast( cell_information[perm_sort[-2]]["num_nodes"], root=0)
        gmsh_facet_id = cell_information[perm_sort[-2]]["id"]
        marked_facets = np.asarray(topologies[gmsh_facet_id]["topology"], dtype=np.int64)
        facet_values = np.asarray(topologies[gmsh_facet_id]["cell_data"], dtype=np.int32)

    cells = np.asarray(topologies[cell_id]["topology"], dtype=np.int64)
    cell_values = np.asarray(topologies[cell_id]["cell_data"], dtype=np.int32)

else:
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([None, None], root=0)
    cells, x = np.empty([0, num_nodes], np.int64), np.empty([0, gdim])
    cell_values = np.empty((0,), dtype=np.int32)
    num_facet_nodes = MPI.COMM_WORLD.bcast(None, root=0)
    marked_facets = np.empty((0, num_facet_nodes), dtype=np.int64)
    facet_values = np.empty((0,), dtype=np.int32)

# Create distributed mesh
ufl_domain = ufl_mesh_from_gmsh(cell_id, gdim)
gmsh_cell_perm = cell_perm_gmsh(to_type(str(ufl_domain.ufl_cell())), num_nodes)
cells = cells[:, gmsh_cell_perm]
mesh = create_mesh(MPI.COMM_WORLD, cells, x[:, :gdim], ufl_domain)
tdim = mesh.topology.dim
fdim = tdim - 1
# Permute facets from MSH to DOLFINx ordering
# FIXME: Last argument is 0 as all facets are the same for tetrahedra
facet_type = cell_entity_type(to_type(str(ufl_domain.ufl_cell())), fdim, 0)
gmsh_facet_perm = cell_perm_gmsh(facet_type, num_facet_nodes)
marked_facets = np.asarray(marked_facets[:, gmsh_facet_perm], dtype=np.int64)

local_entities, local_values = distribute_entity_data(mesh, fdim, marked_facets, facet_values)
mesh.topology.create_connectivity(fdim, tdim)
adj = create_adjacencylist(local_entities)
# Create DOLFINx MeshTags
ft = meshtags_from_entities(mesh, fdim, adj, np.int32(local_values))
ft.name = "Facet tags"

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 5 (Ellipse)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 80%] Meshing curve 9 (Line)
Info    : Done meshing 1D (Wall 0.00271926s, CPU 0.003234s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Simple recombination completed (Wall 0.00281842s, CPU 0.002813s): 128 quads, 46 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.736475, min Q = 0.305378
Info    : Done meshing 2D (Wall 0.00803676s, CPU 0.008396s)
Info    : Refining mesh...
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 5 order 2
Info    : [ 20%] Meshing curve 6 order 2
Info    : [ 40%] Meshing curve 7 order 2
Info    : [ 50%] Meshing curve 8 order 2
Info    : [ 70%] Meshing curve 9 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 0.66321 (0 elements in ]0, 0.2]); wo

# Constants / BC and function space

In [4]:
t = 0
T = 8 #8                    # Final time
dt = 1/4000              # Time step size
num_steps = int(T/dt)
k = Constant(mesh, PETSc.ScalarType(dt))  

v_lagrange2 = VectorElement("Lagrange", mesh.ufl_cell(), 2)
s_lagrange1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
W_element = ufl.MixedElement(v_lagrange2, s_lagrange1)
W = FunctionSpace(mesh, W_element)

fdim = mesh.topology.dim - 1

In [5]:
# Boundary conditions
def u_in_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((2, x.shape[1]))
    values[0, :] = 1.0
    return values

def u_wall_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[PETSc.ScalarType]:
    """Return the zero velocity at the wall."""
    return np.zeros((2, x.shape[1]))

# we use function to define the values for u_in, u_wall and p_out
u_in = dolfinx.fem.Function(W.sub(0).collapse()[0])
u_in.interpolate(u_in_eval)

u_wall = dolfinx.fem.Function(W.sub(0).collapse()[0])
u_wall.interpolate(u_wall_eval)

# then for each tagged edge, we search for the associated d.o.f. with 
# locate_dofs_topological, then impose the dirichlet bc using dirichletbc

# walls
wall_facets = ft.indices[ft.values == obstacle_marker]
bdofs_wall = dolfinx.fem.locate_dofs_topological(
    (W.sub(0), W.sub(0).collapse()[0]), mesh.topology.dim - 1, wall_facets)
wall_bc = dolfinx.fem.dirichletbc(u_wall, bdofs_wall, W.sub(0))

# sides
sides_facets = ft.indices[ft.values == wall_marker]
bdofs_side = dolfinx.fem.locate_dofs_topological(
    (W.sub(0), W.sub(0).collapse()[0]), mesh.topology.dim - 1, sides_facets)
sides_bc = dolfinx.fem.dirichletbc(u_in, bdofs_side, W.sub(0))

# inlet
inlet_facets = ft.indices[ft.values == inlet_marker]
bdofs_inlet = dolfinx.fem.locate_dofs_topological(
    (W.sub(0), W.sub(0).collapse()[0]), mesh.topology.dim - 1, inlet_facets)
inlet_bc = dolfinx.fem.dirichletbc(u_in, bdofs_inlet, W.sub(0))

# list of all BCs
bc = [inlet_bc, sides_bc, wall_bc]

# Week form

In [6]:
# Test and trial functions
vq = ufl.TestFunction(W)
(v, q) = ufl.split(vq)

dup = ufl.TrialFunction(W)
u_n1,p_n1 = ufl.split(dup)


# Define functions for solutions at previous and current time steps
u_np_n = Function(W)
u_n,p_n = u_np_n.split()


# Variational forms (Be careful with the use of grad() or nabla_grad(), one is
# dui/dxj, the other duj/dxi, see documentation)
Re = 100.
nu = Constant(mesh, PETSc.ScalarType((2*r)/Re))

In [7]:
### directly solving navier stokes

F1 = ((1/k)*inner(u_n1 - u_n, v)*dx
    + inner(ufl.grad(u_n) * u_n, v) * dx 
    + nu * inner(ufl.grad(u_n1), ufl.grad(v)) * dx
    + inner(ufl.div(u_n1), q) * dx
    - inner(p_n1, ufl.div(v)) * dx)

a = form(lhs(F1))
L = form(rhs(F1))
A = assemble_matrix(a, bcs=bc)
A.assemble()
b = create_vector(L)  

print(A.getInfo())
print(A.isSymmetric())
print(A.size)

{'block_size': 1.0, 'nz_allocated': 239780.0, 'nz_used': 239780.0, 'nz_unneeded': 0.0, 'memory': 3002236.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}
False
(6080, 6080)


# Solver settings 

In [8]:
### parametres du solver 

solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A)
solver1.setType("preonly")
solver1.setConvergenceHistory()
pc1 = solver1.getPC()
pc1.setType("lu")
pc1.setFactorSolverType("mumps")

# Solving step

In [9]:

# listes des solutions de taille nb pas de temps

list_u = []
list_p = []

### fonction utilisée pour la resolution du problème, Il est impossible d'utiliser les fonctions trials pour la resolution de pb non lineaire

up_temp = Function(W)

# bar de progression du calcul

progress = tqdm.notebook.tqdm(desc="Solving PDE", total=num_steps)

# boucle de resolution

for i in range(num_steps):
    progress.update(1)
    # Update current time step
    t += dt
    # Update inlet velocity
    # inlet_velocity.t = t
    # u_inlet.interpolate(inlet_velocity)
    
    # resolution
    with b.localForm() as loc:
        loc.set(0)
    assemble_vector(b, L)
    apply_lifting(b, [a], [bc])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, bc)
    solver1.solve(b, up_temp.vector)
    up_temp.x.scatter_forward()    
    
    
    # create solutions list for vtk plot
    
    if i%200 == 0 :
        (u_f, p_f) = (up_temp.sub(0).collapse(), up_temp.sub(1).collapse())
        list_u.append(u_f.copy())
        list_p.append(p_f.copy())
        
        if i%1000 == 0 :
        
            residual = A * up_temp.vector - b

            print( f"The relative residual is: {residual.norm() / b.norm()}." )

    # Update variable with solution form this time step
    with up_temp.vector.localForm() as loc_u, u_np_n.vector.localForm() as loc_un:
        loc_u.copy(loc_un)


Solving PDE:   0%|          | 0/32000 [00:00<?, ?it/s]

The relative residual is: 6.107276231682903e-14.
The relative residual is: 1.7095427597072519e-16.
The relative residual is: 1.6861000283020166e-16.
The relative residual is: 1.7017694541769763e-16.
The relative residual is: 1.8859187033131067e-16.
The relative residual is: 1.7086438667016413e-16.
The relative residual is: 1.7828348425947664e-16.
The relative residual is: 1.8452107094958872e-16.
The relative residual is: 1.780644879024149e-16.
The relative residual is: 1.7349976726456252e-16.
The relative residual is: 1.7825307605266943e-16.
The relative residual is: 1.80165348688901e-16.
The relative residual is: 1.696454423154208e-16.
The relative residual is: 1.8302410025960573e-16.
The relative residual is: 1.726678151650126e-16.
The relative residual is: 1.8541962005034038e-16.
The relative residual is: 1.7808573075032488e-16.
The relative residual is: 1.8059073323618266e-16.
The relative residual is: 1.7512284871236503e-16.
The relative residual is: 1.7161008645352786e-16.
The re

# Creating gifs so visualise solutions

In [10]:
!pip install vtk
!pip install pyvista
!pip install pyvirtualdisplay

import os, sys
import pyvista
import vtk
from vtk.util.misc import vtkGetDataRoot 

from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()

plotter = pyvista.Plotter(notebook=True, window_size=(800,600))

plotter.open_gif("u_DNS_direct.gif")

progress = tqdm.notebook.tqdm(desc="creating Gif", total=len(list_u))

for i in list_u: 
    progress.update(1)
    pyvista_cells, cell_types, coordinates = dolfinx.plot.create_vtk_mesh(i.function_space)
    values = i.x.array.reshape(coordinates.shape[0], i.function_space.dofmap.index_map_bs)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, coordinates)
    grid.point_data["u"] = values

    plotter.add_mesh(grid)
    plotter.camera_position = 'xy'
    plotter.render()
    plotter.write_frame()

# Closes and finalizes movie
plotter.close()

display = Display(visible=0, size=(600, 400))
display.start()

plotter = pyvista.Plotter(notebook=True, window_size=(800,600))

plotter.open_gif("p_DNS_direct.gif")

progress = tqdm.notebook.tqdm(desc="creating Gif", total=len(list_u))

for i in list_p: 
    progress.update(1)
    pyvista_cells, cell_types, coordinates = dolfinx.plot.create_vtk_mesh(i.function_space)
    values = i.x.array.reshape(coordinates.shape[0], i.function_space.dofmap.index_map_bs)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, coordinates)
    grid.point_data["p"] = values

    plotter.add_mesh(grid)
    plotter.camera_position = 'xy'
    plotter.render()
    plotter.write_frame()

# Closes and finalizes movie
plotter.close()

creating Gif:   0%|          | 0/160 [00:00<?, ?it/s]

creating Gif:   0%|          | 0/160 [00:00<?, ?it/s]